In [42]:
import os

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import matplotlib.pyplot as plt

import kagglehub

device = torch.device('cpu')
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')

In [10]:
# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

Path to dataset files: /Users/timon/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1


# Preprocessing

## GTSRB Dataset

In [38]:
# Dataset class for GTSRB
class GTSRBDataset(Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.train = train
        
        if self.train:
            # Training data structure in GTSRB
            self.data_dir = os.path.join(root_dir, 'Train')
            self.classes = os.listdir(self.data_dir)
            self.images = []
            self.labels = []
            
            # Load all images and labels
            for class_id in self.classes:
                if not os.path.isdir(os.path.join(self.data_dir, class_id)):
                    continue
                class_dir = os.path.join(self.data_dir, class_id)
                for img_file in os.listdir(class_dir):
                    if img_file.endswith('.png'):
                        self.images.append(os.path.join(class_dir, img_file))
                        self.labels.append(int(class_id))
        else:
            # Test data structure
            self.data_dir = root_dir
            # Load CSV file with test data information
            test_csv = os.path.join(self.data_dir, 'Test.csv')
            df = pd.read_csv(test_csv, sep=',')
            self.images = [os.path.join(self.data_dir, filename) for filename in df['Path']]
            self.labels = df['ClassId'].values
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path)
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

## Data Loaders

In [39]:
# Define transformations
train_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), (0.2672, 0.2564, 0.2629))
])

test_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), (0.2672, 0.2564, 0.2629))
])

# Create datasets and data loaders
def get_data_loaders(root_dir, batch_size=64):
    train_dataset = GTSRBDataset(root_dir=root_dir, train=True, transform=train_transforms)
    test_dataset = GTSRBDataset(root_dir=root_dir, train=False, transform=test_transforms)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    return train_loader, test_loader

# Custom CNN Model

In [40]:
class TrafficSignCNN(nn.Module):
    def __init__(self, num_classes=43):  # GTSRB has 43 classes
        super(TrafficSignCNN, self).__init__()
        
        # First convolutional block
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        # Second convolutional block
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # Third convolutional block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Fourth convolutional block
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.3)
        
        # Fully connected layers - assuming input images are 32x32
        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        # Block 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        
        # Block 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        # Block 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Block 4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        # Flatten
        x = x.view(-1, 256 * 2 * 2)
        
        # Fully connected with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        
        return x

# Evaluation

In [41]:
# Training function
def train_model(model, train_loader, test_loader, epochs=20, learning_rate=0.001):
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    best_accuracy = 0.0
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(test_loader.dataset)
        accuracy = 100 * correct / total
        
        # Adjust learning rate
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')
        
        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_traffic_sign_model.pth')
    
    print(f'Best Accuracy: {best_accuracy:.2f}%')
    return model


# Path should be where you unzipped the Kaggle dataset
dataset_path = path  # Use the path from kagglehub
    
# Get data loaders
train_loader, test_loader = get_data_loaders(dataset_path)
    
# Create model
model = TrafficSignCNN()
    
# Train the model
trained_model = train_model(model, train_loader, test_loader)
    
# Load best model for evaluation
model.load_state_dict(torch.load('best_traffic_sign_model.pth'))

Epoch 1/20:
Train Loss: 1.7385, Val Loss: 0.7251, Accuracy: 76.56%
Epoch 2/20:
Train Loss: 0.4859, Val Loss: 0.2701, Accuracy: 91.35%
Epoch 3/20:
Train Loss: 0.2562, Val Loss: 0.3188, Accuracy: 91.17%
Epoch 4/20:
Train Loss: 0.1727, Val Loss: 0.2464, Accuracy: 92.66%
Epoch 5/20:
Train Loss: 0.1416, Val Loss: 0.1782, Accuracy: 94.57%
Epoch 6/20:
Train Loss: 0.1106, Val Loss: 0.2407, Accuracy: 94.16%
Epoch 7/20:
Train Loss: 0.1110, Val Loss: 0.2174, Accuracy: 94.26%
Epoch 8/20:
Train Loss: 0.0927, Val Loss: 0.1940, Accuracy: 94.54%
Epoch 9/20:
Train Loss: 0.0842, Val Loss: 0.3001, Accuracy: 92.49%
Epoch 10/20:
Train Loss: 0.0405, Val Loss: 0.1751, Accuracy: 95.57%
Epoch 11/20:
Train Loss: 0.0342, Val Loss: 0.1823, Accuracy: 96.03%
Epoch 12/20:
Train Loss: 0.0372, Val Loss: 0.1736, Accuracy: 96.52%
Epoch 13/20:
Train Loss: 0.0375, Val Loss: 0.1528, Accuracy: 96.49%
Epoch 14/20:
Train Loss: 0.0325, Val Loss: 0.1241, Accuracy: 97.17%
Epoch 15/20:
Train Loss: 0.0296, Val Loss: 0.1300, Accura

<All keys matched successfully>